In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
oscars_demography = pd.read_csv('Set/Oscars-demographics-DFE.csv', encoding='latin-1')

In [3]:
oscars_demography.tail()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,birthplace,birthplace:confidence,date_of_birth,date_of_birth:confidence,race_ethnicity,...,award,biourl,birthplace_gold,date_of_birth_gold,movie,person,race_ethnicity_gold,religion_gold,sexual_orientation_gold,year_of_award_gold
436,670946276,False,finalized,3,2/10/15 2:00,"London, England",1.0,7-Mar-71,1.0,White,...,Best Supporting Actress,http://www.nndb.com/people/309/000032213/,NaN,NaN,The Constant Gardener,Rachel Weisz,NaN,NaN,NaN,NaN
437,670946299,False,finalized,3,2/10/15 2:04,"Manchester, England",1.0,20-Oct-56,1.0,White,...,Best Director,http://www.nndb.com/people/887/000044755/,NaN,NaN,Slumdog Millionaire,Danny Boyle,NaN,NaN,NaN,NaN
438,670964912,False,finalized,3,2/10/15 2:03,"Chicago, Il",1.0,26-Jul-22,1.0,White,...,Best Supporting Actor,http://www.nndb.com/people/224/000032128/,NaN,NaN,All the President's Men,Jason Robards,NaN,NaN,NaN,NaN
439,670964956,False,finalized,3,2/10/15 2:01,"Laurel, Ne",1.0,31-Aug-28,1.0,White,...,Best Supporting Actor,http://www.nndb.com/people/764/000022698/,NaN,NaN,Affliction,James Coburn,NaN,NaN,NaN,NaN
440,670972898,False,finalized,3,2/10/15 2:01,"Nevada, Mo",1.0,5-Aug-06,1.0,White,...,Best Director,http://www.nndb.com/people/704/000024632/,NaN,NaN,The Treasure of the Sierra Madre,John Huston,NaN,NaN,NaN,NaN


In [8]:
sc = SparkContext('local[*]')

FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado

In [1]:
import findspark 
findspark.init('C:\Spark\spark-3.0.0-preview2-bin-hadoop2.7')

In [2]:
from pyspark import SparkContext
sc = SparkContext(appName="EstimatePi")

In [3]:
sales = [
    (1, 1, '01', '2017', 500), # sera la minima
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 1, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (4, 3, '01', '2017', 500),
    (4, 3, '01', '2017', 500),
    (4, 3, '02', '2017', 500),
    (4, 3, '03', '2017', 500),    

]

stores = [
    (1 , 'address 1', -1, -1, '30002', 'Georgia'),
    (2 , 'address 2', -2, -2, '30003', 'Georgia'),
    (3 , 'address 2', -3, -3, '30004', 'Georgia'),
    (4 , 'address 2', -4, -4, '10119', 'New York')    
]

In [4]:
salesRdd = sc.parallelize(sales)
storesRdd = sc.parallelize(stores)

In [5]:
# filtramos la informacion de 2017 de ventas y realizamos un cambio de clave
salesToJoin = salesRdd.filter(lambda x: x[3] == '2017')\
    .map(lambda x: (x[1], (x[1],x[4]))) # (K,V) K: id_tienda, V: (id_tienda, total_ventas_mensuales)

# consideramos solo tiendas de Georgia
storesToJoin = storesRdd.filter(lambda x: x[5] == 'Georgia')\
    .map(lambda x: (x[0], x[0]))

In [6]:
# realizamos el join de ambos data frames
# simplificamos el formato para evitar los reduce complejos
# sumarizamos cantidad de ventas por tienda
# buscamos el minimo con reduce
salesToJoin.join(storesToJoin)\
    .map(lambda x: (x[0], x[1][0][1]))\
    .reduceByKey(lambda x, y: x+y)\
    .reduce(lambda x, y: x if x[1] < y[1] else y)

(1, 500)